In [346]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

In [347]:
POSTGRES_USER="ariel"
POSTGRES_PASS="ariel"
POSTGRES_HOST="localhost"
POSTGRES_DB="eco_bikes"
POSTGRES_PORT="5432"
POSTGRES_SCHEMA="eco_bikes"
DB_STR = f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASS}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"

In [348]:
conn = create_engine(DB_STR, connect_args={'options': f'-csearch_path={POSTGRES_SCHEMA}'})

In [349]:
data_scr=pd.read_parquet("/tmp/station_info_eco_bikes.parquet")

In [350]:
data_scr.drop(columns="reload_id",inplace=True)

In [351]:
data_scr.columns

Index(['station_id', 'station_name', 'physical_configuration', 'lat', 'lon',
       'altitude', 'address', 'capacity', 'is_charging_station',
       'nearby_distance', '_ride_code_support'],
      dtype='object')

In [352]:
data_target=pd.read_sql_query("SELECT * FROM eco_bikes.station_info_eco_bikes",con=conn)
data_target.drop(columns="reload_id",inplace=True)
data_target["flag"]=1

In [365]:
data_target

,station_id,station_name,physical_configuration,lat,lon,altitude,address,capacity,is_charging_station,nearby_distance,_ride_code_support,flag
0,3,003 - ADUANA,SMARTLITMAPFRAME,-34.611102,-58.368174,0.0,Moreno & Azopardo,28,False,1000.0,True,1
1,4,004 - Plaza Roma,SMARTLITMAPFRAME,-34.603008,-58.368856,0.0,Av. Corrientes 100,20,False,1000.0,True,1
2,5,005 - Plaza Italia,SMARTLITMAPFRAME,-34.580550,-58.420954,0.0,Av. Sarmiento 2601,42,False,1000.0,True,1
3,6,006 - Parque Lezama,SMARTLITMAPFRAME,-34.628526,-58.369758,0.0,"Avenida Martin Garcia, 295",20,False,1000.0,True,1
4,7,007 - OBELISCO,SMARTLITMAPFRAME,-34.606498,-58.381098,0.0,CARLOS PELEGRINI 215,24,False,1000.0,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...
345,532,072 - INSTITUTO POLITÉCNICO MODELO,SMARTLITMAPFRAME,-34.573033,-58.503908,0.0,Av Constituyentes 5880,20,False,1000.0,True,1
346,533,077 - CRISOLOGO LARRALDE,SMARTLITMAPFRAME,-34.561191,-58.492030,0.0,Galván 3601,20,False,1000.0,True,1
347,534,081 - PARQUE DE LA CIENCIA,SMARTLITMAPFRAME,-34.582963,-58.429979,0.0,Soler 5300,24,False,1000.0,True,1
348,535,090 - HONORIO PUEYRREDON,SMARTLITMAPFRAME,-34.602517,-58.449659,0.0,Honorio Pueyrredón 1653,20,False,1000.0,True,1


## CODE TO MAKE SCD

In [354]:
datetime_now=pd.to_datetime(dt.datetime.now(),format="%Y-%m-%d %HH:%MM:%SS")

In [355]:
data_target_current=data_target[(data_target["flag"]==1)]

In [356]:
df_merge_col=pd.merge(data_scr,data_target_current,on='station_id',how='left')

In [357]:
new_records_filter=pd.isnull(df_merge_col).any(axis=1)


In [358]:
df_new_records = df_merge_col[new_records_filter]

In [359]:
df_new_records_final=df_new_records.copy()
df_new_records_final=df_new_records_final[['station_id',"station_name_x","physical_configuration_x","lat_x","lon_x","altitude_x","address_x","capacity_x","is_charging_station_x","nearby_distance_x","_ride_code_support_x"]]
df_new_records_final.columns=['station_id', 'station_name', 'physical_configuration', 'lat', 'lon','altitude', 'address', 'capacity', 'is_charging_station','nearby_distance', '_ride_code_support']

In [363]:
df_new_records_final['start_date']=datetime_now
df_new_records_final['end_date']="9999-12-30 00:00:00"
df_new_records_final['is_active']=1
df_new_records_final['reload_id']=2

In [366]:
df_new_records_final

,station_id,station_name,physical_configuration,lat,lon,altitude,address,capacity,is_charging_station,nearby_distance,_ride_code_support,start_date,end_date,is_active,reload_id
0,2,002 - Retiro I,SMARTLITMAPFRAME,-34.592424,-58.37471,0.0,AV. Dr. José María Ramos Mejía 1300,40,False,1000.0,True,2023-10-25 16:18:40.258090,9999-12-30 00:00:00,1,2


In [367]:
df_new_records_final.to_sql('station_info_eco_bikes',index=False,con=conn,if_exists="append")

1

In [276]:
df_excluding_new = pd.concat([df_merge_col, df_new_records],axis=0).drop_duplicates(keep=False)

In [277]:
df_scd2_records=df_excluding_new[(df_excluding_new["station_name_x"]!=df_excluding_new["station_name_y"] ) | 
                                 (df_excluding_new["physical_configuration_x"]!=df_excluding_new["physical_configuration_y"]) | 
                                 (df_excluding_new["lat_x"]!=df_excluding_new["lat_y"]) | 
                                 (df_excluding_new["lon_x"]!=df_excluding_new["lon_y"]) | 
                                 (df_excluding_new["altitude_x"]!=df_excluding_new["altitude_y"]) | 
                                 (df_excluding_new["address_x"]!=df_excluding_new["address_y"]) | 
                                 (df_excluding_new["capacity_x"]!=df_excluding_new["capacity_y"]) | 
                                 (df_excluding_new["is_charging_station_x"]!=df_excluding_new["is_charging_station_y"]) | 
                                 (df_excluding_new["nearby_distance_x"]!=df_excluding_new["nearby_distance_y"]) | 
                                 (df_excluding_new["_ride_code_support_x"]!=df_excluding_new["_ride_code_support_y"])]

In [283]:
df_scd2_records_final_replace=df_scd2_records[['station_id',"station_name_y","physical_configuration_y","lat_y","lon_y","altitude_y","address_y","capacity_y","is_charging_station_y","nearby_distance_y","_ride_code_support_y"]]
df_scd2_records_final_replace.columns=['station_id', 'station_name', 'physical_configuration', 'lat', 'lon','altitude', 'address', 'capacity', 'is_charging_station','nearby_distance', '_ride_code_support']

In [280]:
df_scd2_records_final_append=df_scd2_records[['station_id',"station_name_x","physical_configuration_x","lat_x","lon_x","altitude_x","address_x","capacity_x","is_charging_station_x","nearby_distance_x","_ride_code_support_x"]]
df_scd2_records_final_append.columns=['station_id', 'station_name', 'physical_configuration', 'lat', 'lon','altitude', 'address', 'capacity', 'is_charging_station','nearby_distance', '_ride_code_support']

In [284]:
df_scd2_records_final_replace['']

,station_id,station_name,physical_configuration,lat,lon,altitude,address,capacity,is_charging_station,nearby_distance,_ride_code_support
1,3,003 - ADUANA,SMARTLITMAPFRAME,-34.611102,-58.368174,0.0,Moreno & Azopardo,1000.0,False,1000.0,True


In [285]:
df_scd2_records_final_append

,station_id,station_name,physical_configuration,lat,lon,altitude,address,capacity,is_charging_station,nearby_distance,_ride_code_support
1,3,003 - ADUANA,SMARTLITMAPFRAME,-34.611102,-58.368174,0.0,Moreno & Azopardo,28,False,1000.0,True
